In [215]:
import prettytable

available = [3, 3, 2]
max = [
    [7, 5, 3],
    [3, 2, 2],
    [9, 0, 2],
    [2, 2, 2],
    [4, 3, 3]
]
allocation = [
    [0, 1, 0],
    [2, 0, 0],
    [3, 0, 2],
    [2, 1, 1],
    [0, 0, 2]
]
need = [
    [7, 4, 3],
    [1, 2, 2],
    [6, 0, 0],
    [0, 1, 1],
    [4, 3, 1]
]

In [216]:
def show_state():
    table = prettytable.PrettyTable()
    table.field_names = ["Process", "Max", "Allocation", "Need", "Available"]
    for i in range(len(max)):
        table.add_row([
            f"P{i}",
            max[i],
            allocation[i],
            need[i],
            available
        ])
    print("当前系统状态: ")
    print(table)
    
show_state()

当前系统状态: 
+---------+-----------+------------+-----------+-----------+
| Process |    Max    | Allocation |    Need   | Available |
+---------+-----------+------------+-----------+-----------+
|    P0   | [7, 5, 3] | [0, 1, 0]  | [7, 4, 3] | [3, 3, 2] |
|    P1   | [3, 2, 2] | [2, 0, 0]  | [1, 2, 2] | [3, 3, 2] |
|    P2   | [9, 0, 2] | [3, 0, 2]  | [6, 0, 0] | [3, 3, 2] |
|    P3   | [2, 2, 2] | [2, 1, 1]  | [0, 1, 1] | [3, 3, 2] |
|    P4   | [4, 3, 3] | [0, 0, 2]  | [4, 3, 1] | [3, 3, 2] |
+---------+-----------+------------+-----------+-----------+


In [217]:
def checker(available, max, allocation, need):
    work = available.copy()
    finish = [False] * len(max)
    safe_sequence = []

    table = prettytable.PrettyTable()
    table.field_names = [
        "Process",
        "Work",
        "Need",
        "Allocation",
        "Work+Allocation",
        "Finish"
    ]

    while True:
        found = False
        for i in range(len(max)):
            if not finish[i] and all(need[i][j] <= work[j] for j in range(len(work))):
                row = [
                    i,
                    work.copy(),
                    need[i],
                    allocation[i],
                    [work[j] + allocation[i][j] for j in range(len(work))],
                    True
                ]
                for j in range(len(work)):
                    work[j] += allocation[i][j]
                finish[i] = True
                safe_sequence.append(i)
                table.add_row(row)
                found = True
                break
        if not found:
            break

    if all(finish):
        print(table)
        return safe_sequence
    else:
        return False

def request(id, request):
    
    global available, max, allocation, need

    # 检查请求是否小于等于需求
    if any(request[j] > need[id][j] for j in range(len(available))): # type: ignore
        print("请求超过最大需求")
        return False
    
    # 检查请求是否小于等于当前可用资源
    if any(request[j] > available[j] for j in range(len(available))): # type: ignore
        print("请求超过当前可用资源")
        return False

    # 试探性分配资源
    temp_available = available.copy() # type: ignore
    temp_allocation = [row.copy() for row in allocation] # type: ignore
    temp_need = [row.copy() for row in need] # type: ignore

    for j in range(len(available)): # type: ignore
        temp_available[j] -= request[j]
        temp_allocation[id][j] += request[j]
        temp_need[id][j] -= request[j]
        
    print("若分配资源后: ")
    table = prettytable.PrettyTable()
    table.field_names = ["Process", "Max", "Allocation", "Need", "Available"]
    for i in range(len(max)): # type: ignore
        table.add_row([
            f"P{i}",
            max[i], # type: ignore
            temp_allocation[i],
            temp_need[i],
            temp_available
        ])
    print(table)

    result = checker(temp_available, max, temp_allocation, temp_need) # type: ignore
    if result:
        available = temp_available
        allocation = temp_allocation
        need = temp_need
        print(f"安全序列: {result}, 请求被满足并分配资源")
    else:
        print("请求不被满足，系统将进入不安全状态")

In [218]:
checker(available, max, allocation, need)
show_state()

+---------+------------+-----------+------------+-----------------+--------+
| Process |    Work    |    Need   | Allocation | Work+Allocation | Finish |
+---------+------------+-----------+------------+-----------------+--------+
|    1    | [3, 3, 2]  | [1, 2, 2] | [2, 0, 0]  |    [5, 3, 2]    |  True  |
|    3    | [5, 3, 2]  | [0, 1, 1] | [2, 1, 1]  |    [7, 4, 3]    |  True  |
|    0    | [7, 4, 3]  | [7, 4, 3] | [0, 1, 0]  |    [7, 5, 3]    |  True  |
|    2    | [7, 5, 3]  | [6, 0, 0] | [3, 0, 2]  |    [10, 5, 5]   |  True  |
|    4    | [10, 5, 5] | [4, 3, 1] | [0, 0, 2]  |    [10, 5, 7]   |  True  |
+---------+------------+-----------+------------+-----------------+--------+
当前系统状态: 
+---------+-----------+------------+-----------+-----------+
| Process |    Max    | Allocation |    Need   | Available |
+---------+-----------+------------+-----------+-----------+
|    P0   | [7, 5, 3] | [0, 1, 0]  | [7, 4, 3] | [3, 3, 2] |
|    P1   | [3, 2, 2] | [2, 0, 0]  | [1, 2, 2] | [3, 3

In [219]:
request(1, [1, 0, 2])
show_state()

若分配资源后: 
+---------+-----------+------------+-----------+-----------+
| Process |    Max    | Allocation |    Need   | Available |
+---------+-----------+------------+-----------+-----------+
|    P0   | [7, 5, 3] | [0, 1, 0]  | [7, 4, 3] | [2, 3, 0] |
|    P1   | [3, 2, 2] | [3, 0, 2]  | [0, 2, 0] | [2, 3, 0] |
|    P2   | [9, 0, 2] | [3, 0, 2]  | [6, 0, 0] | [2, 3, 0] |
|    P3   | [2, 2, 2] | [2, 1, 1]  | [0, 1, 1] | [2, 3, 0] |
|    P4   | [4, 3, 3] | [0, 0, 2]  | [4, 3, 1] | [2, 3, 0] |
+---------+-----------+------------+-----------+-----------+
+---------+------------+-----------+------------+-----------------+--------+
| Process |    Work    |    Need   | Allocation | Work+Allocation | Finish |
+---------+------------+-----------+------------+-----------------+--------+
|    1    | [2, 3, 0]  | [0, 2, 0] | [3, 0, 2]  |    [5, 3, 2]    |  True  |
|    3    | [5, 3, 2]  | [0, 1, 1] | [2, 1, 1]  |    [7, 4, 3]    |  True  |
|    0    | [7, 4, 3]  | [7, 4, 3] | [0, 1, 0]  |    [7, 

In [220]:
request(4, [3, 3, 0])
show_state()

请求超过当前可用资源
当前系统状态: 
+---------+-----------+------------+-----------+-----------+
| Process |    Max    | Allocation |    Need   | Available |
+---------+-----------+------------+-----------+-----------+
|    P0   | [7, 5, 3] | [0, 1, 0]  | [7, 4, 3] | [2, 3, 0] |
|    P1   | [3, 2, 2] | [3, 0, 2]  | [0, 2, 0] | [2, 3, 0] |
|    P2   | [9, 0, 2] | [3, 0, 2]  | [6, 0, 0] | [2, 3, 0] |
|    P3   | [2, 2, 2] | [2, 1, 1]  | [0, 1, 1] | [2, 3, 0] |
|    P4   | [4, 3, 3] | [0, 0, 2]  | [4, 3, 1] | [2, 3, 0] |
+---------+-----------+------------+-----------+-----------+


In [221]:
request(0, [0, 2, 0])
show_state()

若分配资源后: 
+---------+-----------+------------+-----------+-----------+
| Process |    Max    | Allocation |    Need   | Available |
+---------+-----------+------------+-----------+-----------+
|    P0   | [7, 5, 3] | [0, 3, 0]  | [7, 2, 3] | [2, 1, 0] |
|    P1   | [3, 2, 2] | [3, 0, 2]  | [0, 2, 0] | [2, 1, 0] |
|    P2   | [9, 0, 2] | [3, 0, 2]  | [6, 0, 0] | [2, 1, 0] |
|    P3   | [2, 2, 2] | [2, 1, 1]  | [0, 1, 1] | [2, 1, 0] |
|    P4   | [4, 3, 3] | [0, 0, 2]  | [4, 3, 1] | [2, 1, 0] |
+---------+-----------+------------+-----------+-----------+
请求不被满足，系统将进入不安全状态
当前系统状态: 
+---------+-----------+------------+-----------+-----------+
| Process |    Max    | Allocation |    Need   | Available |
+---------+-----------+------------+-----------+-----------+
|    P0   | [7, 5, 3] | [0, 1, 0]  | [7, 4, 3] | [2, 3, 0] |
|    P1   | [3, 2, 2] | [3, 0, 2]  | [0, 2, 0] | [2, 3, 0] |
|    P2   | [9, 0, 2] | [3, 0, 2]  | [6, 0, 0] | [2, 3, 0] |
|    P3   | [2, 2, 2] | [2, 1, 1]  | [0, 1, 1] | 

In [222]:
request(0, [0, 1, 0])
show_state()

若分配资源后: 
+---------+-----------+------------+-----------+-----------+
| Process |    Max    | Allocation |    Need   | Available |
+---------+-----------+------------+-----------+-----------+
|    P0   | [7, 5, 3] | [0, 2, 0]  | [7, 3, 3] | [2, 2, 0] |
|    P1   | [3, 2, 2] | [3, 0, 2]  | [0, 2, 0] | [2, 2, 0] |
|    P2   | [9, 0, 2] | [3, 0, 2]  | [6, 0, 0] | [2, 2, 0] |
|    P3   | [2, 2, 2] | [2, 1, 1]  | [0, 1, 1] | [2, 2, 0] |
|    P4   | [4, 3, 3] | [0, 0, 2]  | [4, 3, 1] | [2, 2, 0] |
+---------+-----------+------------+-----------+-----------+
+---------+------------+-----------+------------+-----------------+--------+
| Process |    Work    |    Need   | Allocation | Work+Allocation | Finish |
+---------+------------+-----------+------------+-----------------+--------+
|    1    | [2, 2, 0]  | [0, 2, 0] | [3, 0, 2]  |    [5, 2, 2]    |  True  |
|    3    | [5, 2, 2]  | [0, 1, 1] | [2, 1, 1]  |    [7, 3, 3]    |  True  |
|    0    | [7, 3, 3]  | [7, 3, 3] | [0, 2, 0]  |    [7, 

In [223]:
available = [1, 6, 2, 2]
allocation = [
    [0, 0, 3, 2],
    [1, 0, 0, 0],
    [1, 3, 5, 4],
    [0, 3, 3, 2],
    [0, 0, 1, 4]
]
need = [
    [0, 0, 1, 2],
    [1, 7, 5, 0],
    [2, 3, 5, 6],
    [0, 6, 5, 2],
    [0, 6, 5, 6]
]
max = [
    [0, 0, 4, 4],
    [2, 7, 5, 0],
    [3, 6, 10, 10],
    [0, 9, 8, 4],
    [0, 6, 6, 10]
]

show_state()

当前系统状态: 
+---------+----------------+--------------+--------------+--------------+
| Process |      Max       |  Allocation  |     Need     |  Available   |
+---------+----------------+--------------+--------------+--------------+
|    P0   |  [0, 0, 4, 4]  | [0, 0, 3, 2] | [0, 0, 1, 2] | [1, 6, 2, 2] |
|    P1   |  [2, 7, 5, 0]  | [1, 0, 0, 0] | [1, 7, 5, 0] | [1, 6, 2, 2] |
|    P2   | [3, 6, 10, 10] | [1, 3, 5, 4] | [2, 3, 5, 6] | [1, 6, 2, 2] |
|    P3   |  [0, 9, 8, 4]  | [0, 3, 3, 2] | [0, 6, 5, 2] | [1, 6, 2, 2] |
|    P4   | [0, 6, 6, 10]  | [0, 0, 1, 4] | [0, 6, 5, 6] | [1, 6, 2, 2] |
+---------+----------------+--------------+--------------+--------------+


In [224]:
checker(available, max, allocation, need)
if checker:
    print("系统处于安全状态")

+---------+-----------------+--------------+--------------+-----------------+--------+
| Process |       Work      |     Need     |  Allocation  | Work+Allocation | Finish |
+---------+-----------------+--------------+--------------+-----------------+--------+
|    0    |   [1, 6, 2, 2]  | [0, 0, 1, 2] | [0, 0, 3, 2] |   [1, 6, 5, 4]  |  True  |
|    3    |   [1, 6, 5, 4]  | [0, 6, 5, 2] | [0, 3, 3, 2] |   [1, 9, 8, 6]  |  True  |
|    1    |   [1, 9, 8, 6]  | [1, 7, 5, 0] | [1, 0, 0, 0] |   [2, 9, 8, 6]  |  True  |
|    2    |   [2, 9, 8, 6]  | [2, 3, 5, 6] | [1, 3, 5, 4] | [3, 12, 13, 10] |  True  |
|    4    | [3, 12, 13, 10] | [0, 6, 5, 6] | [0, 0, 1, 4] | [3, 12, 14, 14] |  True  |
+---------+-----------------+--------------+--------------+-----------------+--------+
系统处于安全状态


In [225]:
request(2, [1, 2, 2, 2])
show_state()

若分配资源后: 
+---------+----------------+--------------+--------------+--------------+
| Process |      Max       |  Allocation  |     Need     |  Available   |
+---------+----------------+--------------+--------------+--------------+
|    P0   |  [0, 0, 4, 4]  | [0, 0, 3, 2] | [0, 0, 1, 2] | [0, 4, 0, 0] |
|    P1   |  [2, 7, 5, 0]  | [1, 0, 0, 0] | [1, 7, 5, 0] | [0, 4, 0, 0] |
|    P2   | [3, 6, 10, 10] | [2, 5, 7, 6] | [1, 1, 3, 4] | [0, 4, 0, 0] |
|    P3   |  [0, 9, 8, 4]  | [0, 3, 3, 2] | [0, 6, 5, 2] | [0, 4, 0, 0] |
|    P4   | [0, 6, 6, 10]  | [0, 0, 1, 4] | [0, 6, 5, 6] | [0, 4, 0, 0] |
+---------+----------------+--------------+--------------+--------------+
请求不被满足，系统将进入不安全状态
当前系统状态: 
+---------+----------------+--------------+--------------+--------------+
| Process |      Max       |  Allocation  |     Need     |  Available   |
+---------+----------------+--------------+--------------+--------------+
|    P0   |  [0, 0, 4, 4]  | [0, 0, 3, 2] | [0, 0, 1, 2] | [1, 6, 2, 2] |
| 